In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Paper Preprocessing

In [ ]:
# Load and inspect the data
data = pd.read_csv('Data Set 1.csv')
print(data.columns)

Index(['meanA', 'meanAstd', 'meangstd', 'stdmeanA', 'stdstda', 'meang',
       'skewmeanA', 'skewstdA', 'skewG', 'minmeanA', 'minstdA', 'minG',
       'maxmeanA', 'maxstdA', 'maxmG', 'kurtmeanA', 'kurtstdA', 'kurtG',
       'entromeanA', 'entrostdA', 'entroG', 'iqrmeanA', 'iqrstdA', 'iqrG',
       'class'],
      dtype='object')


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Load the data
data = pd.read_csv('Data Set 1.csv')

# Separate features and labels
features = data.drop(columns=['class'])
labels = data['class']

# Standardize features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Apply SMOTE to balance the dataset
smote = SMOTE()
features_resampled, labels_resampled = smote.fit_resample(features_scaled, labels)

# Save the oversampled dataset
oversampled_data = pd.DataFrame(features_resampled, columns=features.columns)
oversampled_data['class'] = labels_resampled
oversampled_data.to_csv('Data Set 1 OverSampled.csv', index=False)

print("Data preprocessing and oversampling completed successfully.")

Data preprocessing and oversampling completed successfully.


## 1 CSV

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
import joblib

# Step 3: Load the data from Google Drive
# Make sure to place "Data Set 1 OverSampled.csv" in a known directory in your Google Drive
data_path = 'Data Set 1 OverSampled.csv'
df = pd.read_csv(data_path)

# Step 4: Preprocess the data
X = df.drop('class', axis=1)  # Assuming 'class' is the column name for labels
y = df['class']

# Step 5: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Cross-validation and Hyperparameter Tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# Step 7: Evaluate the best model using cross-validation
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)
print(f'Cross-validation accuracy: {cv_scores.mean()}')

# Step 8: Test the model on the testing set
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy on the test data: {test_accuracy}')

# Step 9: Save the best model to Google Drive
model_path = '/content/drive/My Drive/anomaly_detection_model.joblib'
joblib.dump(best_model, model_path)
print(f'Model saved to {model_path}')

##2 CSV

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

# Step 3: Load the data from Google Drive
# Make sure to place "Data Set 1 OverSampled.csv" and "Data Set 2.csv" in known directories in your Google Drive
data_path1 = 'Data Set 1 OverSampled.csv'
data_path2 = 'Data Set 2 OverSampled.csv'

df1 = pd.read_csv(data_path1)
df2 = pd.read_csv(data_path2)

# Step 4: Combine the data
df_combined = pd.concat([df1, df2])

# Step 5: Preprocess the data
X = df_combined.drop('class', axis=1)  # Assuming 'class' is the column name for labels
y = df_combined['class']

# Step 6: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 8: Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Step 9: Save the model to Google Drive
model_path = '/content/drive/My Drive/anomaly_detection_model.joblib'
joblib.dump(model, model_path)
print(f'Model saved to {model_path}')

Accuracy: 0.8861788617886179
Model saved to /content/drive/My Drive/anomaly_detection_model.joblib


##Test

In [ ]:
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score

# Step 3: Load the saved model from Google Drive
model_path = '/content/drive/My Drive/anomaly_detection_model.joblib'
model = joblib.load(model_path)

# Step 4: Load the new test data from Google Drive
# Make sure to place "New_Test_Data.csv" in a known directory in your Google Drive
test_data_path = 'Data Set 1 OverSampled.csv'
test_df = pd.read_csv(test_data_path)

# Step 5: Preprocess the test data
# Assuming 'class' is the column name for labels in the test data, if not available, remove the next line
X_test = test_df.drop('class', axis=1)  # Features
y_test = test_df['class']  # Actual labels

# Step 6: Make predictions
y_pred = model.predict(X_test)

# Step 7: Evaluate predictions (if actual labels are available)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy on the test data: {accuracy}')

Accuracy on the test data: 0.9755952380952381


##Paper way to train the model

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
import joblib

# Load the dataset
dataset_path = '/content/Data Set 1 OverSampled.csv'
data = pd.read_csv(dataset_path)

# Function to extract features and ensure numeric values
def extract_features(df):
    features = {}
    features['mean'] = df.mean().mean()
    features['std'] = df.std().mean()
    features['skewness'] = df.apply(skew).mean()
    features['kurtosis'] = df.apply(kurtosis).mean()
    features['max'] = df.max().max()
    features['min'] = df.min().min()
    return features

# Apply sliding window
window_size = 100  # assuming 100 Hz sample rate, 1 second window
step_size = window_size // 2  # 50% overlap

feature_list = []

for start in range(0, len(data) - window_size + 1, step_size):
    window = data.iloc[start:start + window_size]
    features = extract_features(window)
    feature_list.append(features)

# Convert list of feature dicts to DataFrame
features_df = pd.DataFrame(feature_list)

# Extract target variable
target = data['class'][window_size//2::step_size][:len(features_df)]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features_df, target, test_size=0.2, random_state=42)

# Train Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Cross-validation
cv_scores = cross_val_score(clf, X_train, y_train, cv=10)

# Save the trained model
model_path = '/content/drive/MyDrive/New_Dataset.pkl'
joblib.dump(clf, model_path)

# Output results
cv_scores.mean(), cv_scores.std(), model_path

/usr/local/lib/python3.10/dist-packages/pandas/core/apply.py:814: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  results[i] = self.f(v)


(0.95, 0.15, '/content/drive/MyDrive/New_Dataset.pkl')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import joblib

# Load the dataset
data = pd.read_csv('/content/Data Set 1 OverSampled.csv')

# Inspect the dataset to confirm the column names
print(data.head())
print(data.columns)

# Assuming these are the correct columns for accelerometer and gyroscope statistics
# The column names are adapted based on the provided column names in your dataset
feature_columns = [
    'meanA', 'meanAstd', 'meangstd', 'stdmeanA', 'stdstda', 'meang',
    'skewmeanA', 'skewstdA', 'skewG', 'minmeanA', 'minstdA', 'minG',
    'maxmeanA', 'maxstdA', 'maxmG', 'kurtmeanA', 'kurtstdA', 'kurtG',
    'entromeanA', 'entrostdA', 'entroG', 'iqrmeanA', 'iqrstdA', 'iqrG'
]

# Prepare the feature matrix and labels
X = data[feature_columns]
y = data['class']  # Assuming 'class' is the column with labels

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Save the trained model
joblib.dump(model, 'anomaly_detection_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

      meanA  meanAstd  meangstd  stdmeanA   stdstda     meang  skewmeanA  \
0  1.017204  0.058578  0.642445  0.053185  0.048299  0.526831   0.934144   
1  1.003853  0.005816  0.168383  0.003261  0.002917  0.207648  -0.410724   
2  1.003775  0.023734  0.368631  0.025993  0.025151  0.368631   1.852045   
3  1.004330  0.016759  0.279067  0.014712  0.009971  0.255733   0.474464   
4  1.004302  0.009317  0.271160  0.002629  0.003489  0.233640   0.456424   

   skewstdA     skewG  minmeanA  ...  kurtmeanA  kurtstdA     kurtG  \
0  1.157455  1.658387  0.928243  ...   0.579301  0.791138  3.657740   
1  0.751071  0.904551  0.990956  ...   0.459438 -0.048304  0.222102   
2  2.815163  0.596992  0.967968  ...   4.331743  8.655530 -0.432390   
3  0.757765  0.330055  0.984220  ...   0.851071  0.266029 -0.800815   
4  0.464779  0.607188  0.999489  ...  -0.543562  0.111179 -0.785866   

   entromeanA  entrostdA    entroG  iqrmeanA   iqrstdA      iqrG  class  
0    4.604300   4.301860  4.511478  0.0337

['scaler.pkl']

##Test Research paper

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.metrics import accuracy_score, f1_score
import joblib

# Load the trained model
model_path = '/content/drive/MyDrive/New_Dataset.pkl'
clf = joblib.load(model_path)

# Load the new dataset
new_dataset_path = '/content/Data Set 2 OverSampled.csv'  # Replace with the path to your new dataset
new_data = pd.read_csv(new_dataset_path)

# Function to extract features and ensure numeric values
def extract_features(df):
    features = {}
    features['mean'] = df.mean().mean()
    features['std'] = df.std().mean()
    features['skewness'] = df.apply(skew).mean()
    features['kurtosis'] = df.apply(kurtosis).mean()
    features['max'] = df.max().max()
    features['min'] = df.min().min()
    return features

# Apply sliding window
window_size = 100  # assuming 100 Hz sample rate, 1 second window
step_size = window_size // 2  # 50% overlap

feature_list = []

for start in range(0, len(new_data) - window_size + 1, step_size):
    window = new_data.iloc[start:start + window_size]
    features = extract_features(window)
    feature_list.append(features)

# Convert list of feature dicts to DataFrame
new_features_df = pd.DataFrame(feature_list)

# Extract target variable
new_target = new_data['class'][window_size//2::step_size][:len(new_features_df)]

# Predict using the loaded model
predictions = clf.predict(new_features_df)

# Evaluate the model's performance on the new dataset
accuracy = accuracy_score(new_target, predictions)
f1 = f1_score(new_target, predictions, average='weighted')

# Output results
accuracy, f1

/usr/local/lib/python3.10/dist-packages/pandas/core/apply.py:814: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  results[i] = self.f(v)


(0.9230769230769231, 0.9226190476190476)